In [4]:
# Importing the libraries needed
import os
import sys
sys.path.append("/Users/leonardovida/dev/HistAware")

import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, LoggingHandler, util
from transformers import BertTokenizer, BertModel

from src import iterators

2020-10-12 15:52:38,125 — parsers — DEBUG — Test message
2020-10-12 15:52:38,125 — parsers — DEBUG — Test message
2020-10-12 15:52:38,127 — iterators — DEBUG — Test message


In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Sentence Transformer 

In [5]:
# Setup the model - Bertje
tokenizer = BertTokenizer.from_pretrained("wietsedv/bert-base-dutch-cased")
model = BertModel.from_pretrained("wietsedv/bert-base-dutch-cased")

In [18]:
# Setup the file
csv = iterators.iterate_directory("../data/processed/selected_articles/", ".csv")
df = pd.concat([pd.read_csv(c["article_path"]) for c in csv],ignore_index=True)
df.sort_values(by=["count"], ascending=False, inplace=True)

In [19]:
df_selected = df[0:100]
df_remaining = df[101:]

In [20]:
embedder = SentenceTransformer(
        "../data/models/distiluse/distiluse-base-multilingual-cased"
    )

In [21]:
corpus = list(df_remaining["text"])
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [22]:
queries = list(df_selected["text"])
queries_embeddings = embedder.encode(queries, convert_to_tensor=True)

In [23]:
results = []
for query_emb in queries_embeddings:
    test = util.semantic_search(
        query_embeddings=query_emb,
        corpus_embeddings=corpus_embeddings,
        query_chunk_size=100,
        corpus_chunk_size=100000,
        top_k=10,
    )
    results.append(test)
    test = []


In [24]:
results[1]

[[{'corpus_id': 56, 'score': 1.0000002},
  {'corpus_id': 1191, 'score': 0.46423584},
  {'corpus_id': 1001, 'score': 0.46423584},
  {'corpus_id': 169, 'score': 0.46423584},
  {'corpus_id': 4795, 'score': 0.4443941},
  {'corpus_id': 1085, 'score': 0.4443941},
  {'corpus_id': 2554, 'score': 0.4293515},
  {'corpus_id': 887, 'score': 0.42333776},
  {'corpus_id': 13, 'score': 0.42276603},
  {'corpus_id': 3721, 'score': 0.41876274}]]

In [25]:
df_selected.iloc[1]["text"]

'ning tijdens de ontgassing normaal kan doorgaan. In de Belgische mijn „Le Grand Trait" te Frameries in Henegouwen „oogstte" men op deze wijze in 2 maanden tijds 378.000 m 3 methaangas, in de mijn „Saint Albert" te Ressaix in een iets langere periode 428.650 m 3 methaan. In Henegouwen wordt het gas reeds naar buiten geleverd via de lichtgasfabrieken te Tertre. Methaangas levert 8000 tot 9000 caloriën warmte, hetgeen tweemaal zoveel is als gewoon cokesovengas. In vele andere mijnen, waaronder de Kempische, neemt men proeven. Er bestaan plannen in Belgisch Limburg een leidingermet aan te leggen voor de distributie van het gas aan de bevolking. Een probleem vormt echter de vrij onregelmatige toevoer, waarmee men ongetwijfeld te kampen zal krijgen. In de mijn Hirschbach in het Saargebied heeft men een andere methode gevolgd. In deze mijn ontsnapte zoveel gas dat met luchtverversing niet voldoende te bereiken was. Een gedeelte van de mijn werd daarom met dammen van de rest afgesloten. Door 

In [27]:
df_remaining.iloc[56]["text"]

'ning tijdens de ontgassing normaal kan doorgaan. In de Belgische mijn „Le Grand Trait" te Frameries in Henegouwen „oogstte" men op deze wijze in 2 maanden tijds 378.000 m 3 methaangas, in de mijn „Saint Albert" te Ressaix in een iets langere periode 428.650 m 3 methaan. In Henegouwen wordt het gas reeds naar buiten geleverd via de lichtgasfabrieken te Tertre. Methaangas levert 8000 tot 9000 caloriën warmte, hetgeen tweemaal zoveel is als gewoon cokesovengas. In vele andere mijnen, waaronder de Kempische, neemt men proeven. Er bestaan plannen in Belgisch Limburg een leidingermet aan te leggen voor de distributie van het gas aan de bevolking. Een probleem vormt echter de vrij onregelmatige toevoer, waarmee men ongetwijfeld te kampen zal krijgen. In de mijn Hirschbach in het Saargebied heeft men een andere methode gevolgd. In deze mijn ontsnapte zoveel gas dat met luchtverversing niet voldoende te bereiken was. Een gedeelte van de mijn werd daarom met dammen van de rest afgesloten. Door 

# For sentiment analysis system

In [36]:
df = pd.read_csv("../data/processed/selected_articles/2020-10-07_aardgas.csv")
# This step is necessary to modify the classes into a `list` column
# To be used downstream
#df["list"] = df[df.columns["text", "type"]].values.tolist()
new_df = df[["text", "type"]].copy()
new_df.head()

,text,type
0,Aardgas bij Oostzaan,title
1,Op een diepte van 220 meter is Dinsdagmiddag b...,p
2,Vier nieuwe boringen in het Duitse petro leumg...,p
3,"'s-GRAVENHAGE, 16 Nov. — Aan de Naamloze Venno...",p
4,Onmiddellijk na deze toespraken werd weer inge...,p


In [41]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = "max_length"
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
MODEL = "wietsedv/bert-base-dutch-cased"

In [33]:
# Setup the model - Bertje
tokenizer = BertTokenizer.from_pretrained(MODEL)
model = BertModel.from_pretrained(MODEL)

Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of dataloaded to the memory and then passed to the neural network needs to be controlled.

In [45]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.type
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }


In [46]:

# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (2565, 2)
TRAIN Dataset: (2052, 2)
TEST Dataset: (513, 2)


In [47]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
            'shuffle': True,
            'num_workers': 0
            }

test_params = {'batch_size': VALID_BATCH_SIZE,
            'shuffle': True,
            'num_workers': 0
            }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [31]:
encoded_input = tokenizer(batch_sentences,
    # Pad to the longest allowed length by the model 
    padding="max_length",
    # Truncate to maximum length
    truncation=True,
    return_tensors="pt")
tokenizer.decode(encoded_input["input_ids"])

NameError: name &#39;batch_sentences&#39; is not defined

# Correct OCR words

In [130]:

import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import nl_core_news_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import spacy
import enchant

import torch
import re
import nltk
from enchant.checker import SpellChecker
from difflib import SequenceMatcher
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
from transformers import AutoTokenizer, AutoModel


In [166]:
text_original = a
text = a

In [171]:
rep = { '\n': ' ', '\\': ' ', '\"': '"', '-': ' ', '"': ' " ', 
        '"': ' " ', '"': ' " ', ',':' , ', '.':' . ', '!':' ! ', 
        '?':' ? ', "n't": " not" , "'ll": " will", '*':' * ', 
        '(': ' ( ', ')': ' ) ', "s'": "s '"}
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], text)

def get_personslist(text):
    personslist=[]
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if isinstance(chunk, nltk.tree.Tree) and chunk.label() == 'PERSON':
                personslist.insert(0, (chunk.leaves()[0][0]))
    return list(set(personslist))

personslist = get_personslist(text)

ignorewords = personslist + ["!", ",", ".", "\"", "?", '(', ')', '*', "'"]
#using enchant.checker.SpellChecker, identify incorrect words
d = SpellChecker("nl_NL")
words = text.split()
incorrectwords = [w for w in words if not d.check(w) and w not in ignorewords]
# using enchant.checker.SpellChecker, get suggested replacements
suggestedwords = [d.suggest(w) for w in incorrectwords]
# replace incorrect words with [MASK]
for w in incorrectwords:
    text = text.replace(w, '[MASK]')
    text_original = text_original.replace(w, '[MASK]')
    
print(text)

+ Op [MASK] Heren Hemelvaartsdag overleed tot onze diepe droefheid   ,    na een langdurig lijden op [MASK] jarige leeft   ,    voorzien van de laatste H   .   H   .    Sacramenten der Stervenden   ,    en in volledige overgave aan Gods H   .    [MASK] [MASK] [MASK] weduwnaar van Mextildns [MASK] Wij bevelen [MASK] ziel in Uwe [MASK]   .    [MASK] aan   .    De bedroefde [MASK] Hnbert [MASK] [MASK] en kinderen Jan [MASK] Wetsels en kinderen Peter Wetsels en kinderen Jan Paas Wetsels en kinderen Charel [MASK] Wetsels en kind Jac   .    Wetsels Brands en kind De plechtige [MASK]   .   dienst gevolgd door de begrafenis zal plaats hebben Maandag [MASK] Mei om 9   .   [MASK] uur in de par   .    kerk te [MASK]   .    [MASK]   .    ten sterf [MASK] om 9 uur   .    [MASK] Zondagavond om 7 uur   .    i &gt; i Voor Uwe [MASK] en vele blijken Van deelneming   ,    betoond bij de ziekte   ,    het overlijden en de begrafenis van onze dierbare broer   ,    zwager   ,    oom en neef   ,    de [MASK

In [168]:

# Load, train and predict using pre-trained model
tokenizer = AutoTokenizer.from_pretrained("../data/models/bertje")
tokenized_text = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
MASKIDS = [i for i, e in enumerate(tokenized_text) if e == '[MASK]']
# Create the segments tensors
segs = [i for i, e in enumerate(tokenized_text) if e == "."]
segments_ids=[]
prev=-1
for k, s in enumerate(segs):
    segments_ids = segments_ids + [k] * (s-prev)
    prev=s
segments_ids = segments_ids + [len(segs)] * (len(tokenized_text) - len(segments_ids))
segments_tensors = torch.tensor([segments_ids])
# prepare Torch inputs 
tokens_tensor = torch.tensor([indexed_tokens])
# Load pre-trained model
model = AutoModel.from_pretrained("../data/models/bertje")
# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

In [169]:

#Predict words for mask using BERT; 
def predict_word(text_original, predictions, maskids):
    pred_words=[]
    for i in range(len(MASKIDS)):
        preds = torch.topk(predictions[0, MASKIDS[i]], k=50) 
        indices = preds.indices.tolist()
        list1 = tokenizer.convert_ids_to_tokens(indices)
        list2 = suggestedwords[i]
        simmax=0
        predicted_token=''
        for word1 in list1:
            for word2 in list2:
                s = SequenceMatcher(None, word1, word2).ratio()
                if s is not None and s > simmax:
                    simmax = s
                    predicted_token = word1
        text_original = text_original.replace('[MASK]', predicted_token, 1)
    return text_original

text_original = predict_word(text_original, predictions, MASKIDS)
print (text_original)

IndexError: index 2 is out of bounds for dimension 0 with size 1

# LDA model and TF-IDF

In [74]:
doc = df["text"]

In [79]:
nlp = nl_core_news_lg.load()

# My list of stop words.
#stop_list = ["Mrs.","Ms.","say","WASHINGTON","'s","Mr.",]

# Updates spaCy's default stop words list with my additional words. 
#nlp.Defaults.stop_words.update(stop_list)

# Iterates over the words in the stop words list and resets the "is_stop" flag.
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [80]:

def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# The add_pipe function appends our functions to the default pipeline.
nlp.add_pipe(lemmatizer,name='lemmatizer',after='ner')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [81]:
doc_list = []
# Iterates through each article in the corpus.
for d in tqdm(doc):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(d)
    doc_list.append(pr)

In [82]:
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [83]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=10, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [94]:
lda_model.print_topics()

[(0,
  &#39;0.007*&quot;3&quot; + 0.007*&quot;b&quot; + 0.006*&quot;u.&quot; + 0.006*&quot;ned&quot; + 0.005*&quot;100&quot; + 0.005*&quot;4&quot; + 0.005*&quot;n&quot; + 0.005*&quot;1&quot; + 0.005*&quot;10&quot; + 0.005*&quot;eva&quot;&#39;),
 (1,
  &#39;0.008*&quot;komen&quot; + 0.007*&quot;twee&quot; + 0.006*&quot;rust&quot; + 0.006*&quot;goed&quot; + 0.006*&quot;minuut&quot; + 0.006*&quot;doelpunt&quot; + 0.005*&quot;gast&quot; + 0.005*&quot;brengen&quot; + 0.005*&quot;weten&quot; + 0.004*&quot;gaan&quot;&#39;),
 (2,
  &#39;0.016*&quot;ƒ&quot; + 0.015*&quot;1&quot; + 0.012*&quot;2&quot; + 0.011*&quot;■&quot; + 0.010*&quot;tel&quot; + 0.010*&quot;f&quot; + 0.009*&quot;koop&quot; + 0.009*&quot;ét&quot; + 0.009*&quot;no.&quot; + 0.008*&quot;br&quot;&#39;),
 (3,
  &#39;0.007*&quot;komen&quot; + 0.006*&quot;goed&quot; + 0.006*&quot;groot&quot; + 0.005*&quot;één&quot; + 0.005*&quot;gaan&quot; + 0.005*&quot;jaar&quot; + 0.004*&quot;zien&quot; + 0.004*&quot;zeggen&quot; + 0.004*&quot;make